# Final Project of Applied Data Science Capstone
# - Find a new living area in Toronto

## 1. Introduction
Based on the results of previous weeks, we have the data about the venues in Toronto. Now it's time to help David to decide where to move so he can start looking for apartments in that area.
David is a software engineer and he has a car. Normally, he works from home, but he sometimes needs to go to the office for meetings. Therefore, the distance between his home and office is not a problem. What really matters is the the quality of life he will have in the new neighborhood. 
David likes to taste coffee and he is also a workout fan since college. In addition, he doesn't like to cook because he hates the cooking smell inside the house, so it would be the best if there are tons of restaurants right down the street. 